根据phage-host信息生成host_json文件

In [14]:
from tqdm import tqdm
import json
input = 'virushostdb_output_filtered.tsv'
host_dict = {}
num_lines = sum(1 for line in open(input, 'r'))-1
with open(input,mode='r',encoding='utf-8') as f:
    f.readline()
    for line in tqdm(f,total=num_lines):
        line_list=line.strip().split('\t')
        # print(len(line_list))
        find_lineage=line_list[14]
        find_taxrank=line_list[13]
        find_taxid=line_list[12]
        assembly_level=line_list[11]
        assembly_category=line_list[10]
        new_gcf=line_list[9]
        gcfs=line_list[8]
        find_name=line_list[7]
        host_tax_rank=line_list[6]
        host_lineage=line_list[5]
        host_taxid=line_list[4]
        report_host=line_list[3]
        if report_host not in host_dict:
            host_dict[report_host] = {
                'host_taxid': host_taxid,
                'host_tax_rank': host_tax_rank,
                'host_lineage': host_lineage,
                'find_name':find_name,
                'gcfs':gcfs,
                'new_gcf':new_gcf,
                'assembly_category':assembly_category,
                'assembly_level':assembly_level,
                'find_taxid':find_taxid,
                'find_taxrank':find_taxrank,
                'find_lineage':find_lineage,
                }
js = json.dumps(host_dict,indent=4)
# fp = open('virushostdb_output_host.json', 'w')
# fp.write(js)
# fp.close()


100%|██████████| 4577/4577 [00:00<00:00, 125671.52it/s]


In [15]:
import pandas as pd
table=[]
line_dict={}
for host_name in host_dict.keys():
    line_dict={"host_name":host_name}
    line_dict.update(host_dict[host_name])
    table.append(line_dict)
    
df=pd.DataFrame(table)
# df.to_csv('virushostdb_output_host.tsv', sep='\t', index=False)


In [3]:
len(df)

602

In [4]:
df["new_gcf"].value_counts()

GCF_000008865.2    25
GCF_000006945.2     8
GCF_022557275.1     4
GCF_000240185.1     4
GCF_008632635.1     4
                   ..
GCF_014844295.1     1
GCF_002983865.1     1
GCF_001886435.1     1
GCF_001028665.1     1
GCF_000832845.1     1
Name: new_gcf, Length: 496, dtype: int64

In [4]:
df["find_taxrank"].value_counts()

species            386
strain             138
no rank             41
subspecies          30
serogroup            4
serotype             1
forma specialis      1
isolate              1
Name: find_taxrank, dtype: int64

In [7]:
df_name = df[["find_name", "new_gcf", "assembly_category", "assembly_level",
    "find_taxid", "find_taxrank", "find_lineage"]]
name_dict = {}

for i in range(df_name.shape[0]):
    find_name = df_name.iloc[i, 0]
    if find_name not in name_dict:
        name_dict[find_name]={
            'new_gcf':df_name.iloc[i,1],
            'assembly_category':df_name.iloc[i,2],
            'assembly_level':df_name.iloc[i,3],
            'find_taxid':df_name.iloc[i,4],
            'find_taxrank':df_name.iloc[i,5],
            'find_lineage':df_name.iloc[i,6]
        }
    else:
        continue

js = json.dumps(name_dict, indent=4)
fp = open('virushostdb_output_findname.json', 'w')
fp.write(js)
fp.close()

In [9]:
table = []
line_dict = {}
for find_name in name_dict.keys():
    line_dict = {"find_name": find_name}
    line_dict.update(name_dict[find_name])
    table.append(line_dict)

df_find = pd.DataFrame(table)
df_find.to_csv('virushostdb_output_find.tsv', sep='\t', index=False)


In [11]:
df_find["new_gcf"].value_counts()

GCF_000006945.2    4
GCF_000011125.1    2
GCF_000006765.1    2
GCF_000009045.1    2
GCF_000009085.1    2
                  ..
GCF_013282745.1    1
GCF_000009485.1    1
GCF_002905815.2    1
GCF_001941405.1    1
GCF_000832845.1    1
Name: new_gcf, Length: 496, dtype: int64

## 构建gcf json文件

In [21]:
df_gcf=df[["new_gcf", "find_name", "assembly_category", "assembly_level"]]


gcf_dict = {}

for i in range(df_gcf.shape[0]):
    gcf = df_gcf.iloc[i, 0]
    if gcf not in gcf_dict:
        gcf_dict[gcf]={
            'find_name':[df_gcf.iloc[i,1]],
            'assembly_category':df_gcf.iloc[i,2],
            'assembly_level':df_gcf.iloc[i,3],
        }
    else:
        find_name=df_gcf.iloc[i,1]
        if find_name not in gcf_dict[gcf]['find_name']:
            gcf_dict[gcf]['find_name'].append(find_name)

js = json.dumps(gcf_dict, indent=4)
fp = open('virushostdb_output_gcf.json', 'w')
fp.write(js)
fp.close()


In [31]:
gcf_output=open('virushostdb_output_gcf(duplicated).tsv',mode="w")
print(len(gcf_dict))
for gcf in gcf_dict:
    length=len(gcf_dict[gcf]['find_name'])
    if length >1:
        gcf_output.write(f"{gcf}\t{length}\t{gcf_dict[gcf]['find_name']}\n")
gcf_output.close()

496


转化为表格

In [22]:
table = []
line_dict = {}
for gcf in gcf_dict.keys():
    line_dict = {"gcf": gcf}
    line_dict.update(gcf_dict[gcf])
    table.append(line_dict)

df_gcf = pd.DataFrame(table)
df_gcf.to_csv('virushostdb_output_gcf.tsv', sep='\t', index=False)


In [8]:
df['find_taxrank'].value_counts()

species            311
strain             124
no rank             38
subspecies          17
serogroup            4
forma specialis      1
isolate              1
Name: find_taxrank, dtype: int64

In [9]:
df_=pd.read_table('virushostdb(non_scaffold)_gcf.tsv')
df_['find_taxrank'].value_counts()


species            316
strain             162
subspecies          17
serogroup            4
forma specialis      1
isolate              1
Name: find_taxrank, dtype: int64